In [7]:
%pip install tensorflow

Note: you may need to restart the kernel to use updated packages.


In [6]:
%pip install imblearn

  Using cached imblearn-0.0-py2.py3-none-any.whl.metadata (355 bytes)
Using cached imblearn-0.0-py2.py3-none-any.whl (1.9 kB)

   ---------------------------------------- 0/3 [sklearn-compat]
   ------------- -------------------------- 1/3 [imbalanced-learn]
   ------------- -------------------------- 1/3 [imbalanced-learn]
   ------------- -------------------------- 1/3 [imbalanced-learn]
   ------------- -------------------------- 1/3 [imbalanced-learn]
   ------------- -------------------------- 1/3 [imbalanced-learn]
   ------------- -------------------------- 1/3 [imbalanced-learn]
   ------------- -------------------------- 1/3 [imbalanced-learn]
   ------------- -------------------------- 1/3 [imbalanced-learn]
   ------------- -------------------------- 1/3 [imbalanced-learn]
   ------------- -------------------------- 1/3 [imbalanced-learn]
   ------------- -------------------------- 1/3 [imbalanced-learn]
   ------------- -------------------------- 1/3 [imbalanced-learn]
   -

In [11]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay, classification_report
from imblearn.over_sampling import SMOTE
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.regularizers import l2
from tensorflow.keras import regularizers

In [16]:
pd.set_option('display.max_columns', None)  # Mostrar todas las columnas en la salida

# Cargar los datos preprocesados
df = pd.read_csv("../data/datos_preprocesados.csv")

df.head()

,fumador,actividad_fisica,vive,edad,sexo,nivel_educativo,ingresos_mensuales,gastos_salud,deudas,diabetes,hipertension,obesidad,cancer,enfermedad_cardiaca,epoc,glucosa,colesterol,trigliceridos,hemoglobina,leucocitos,plaquetas,creatinina,mut_BRCA1,mut_TP53,mut_EGFR,mut_KRAS,mut_PIK3CA,mut_ALK,mut_BRAF,estado_civil_Casado,estado_civil_Divorciado,estado_civil_Soltero,estado_civil_Viudo,tipo_empleo_Autónomo,tipo_empleo_Desempleado,tipo_empleo_Privado,tipo_empleo_Público,tipo_empleo_Temporal,ocupacion_Autónomo,ocupacion_Desempleado,ocupacion_Empleado,ocupacion_Jubilado,continente_nacimiento_América,continente_nacimiento_Antártida,continente_nacimiento_Asia,continente_nacimiento_Europa,continente_nacimiento_Oceanía,continente_nacimiento_África
0,1.0,-1.039961,1.0,0.750106,0,-0.44961,1.499543,2.487493,0.0,1.0,1.0,1.0,0.0,1.0,0.0,1.125931,1.116180,0.518919,-0.262543,-0.924342,0.202956,-0.323535,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
1,0.0,0.344623,0.0,-1.028843,1,0.44846,2.557298,-0.885562,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-1.549388,-1.768373,-1.357231,0.177002,-0.083060,0.709850,-0.883740,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
2,1.0,-1.039961,1.0,1.711700,1,-1.34768,1.082786,-0.885562,0.0,0.0,0.0,0.0,0.0,0.0,1.0,-0.436864,-0.699218,-2.059947,-0.742047,0.481094,-0.116266,2.570857,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
3,1.0,-1.039961,0.0,1.182823,1,-0.44961,-0.009775,2.487493,0.0,1.0,0.0,1.0,1.0,1.0,0.0,-0.258148,0.287809,-0.946194,-0.552243,1.490633,-0.122279,1.030293,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
4,1.0,0.344623,1.0,-0.740365,0,1.34653,0.011400,-0.885562,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-1.940861,-1.565293,-2.566843,-1.161613,1.134325,-0.389633,-0.743689,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0


In [17]:
# Asumimos que el DataFrame df ya existe y contiene la columna 'vive'
# Calculamos la correlación de todas las columnas numéricas con 'vive'
correlaciones = df.corr(numeric_only=True)['vive'].sort_values(key=abs, ascending=False)

# Mostramos las 15 columnas con mayor correlación (positiva o negativa) con 'vive'
correlaciones_mas_fuertes = correlaciones.drop('vive').head(51)
correlaciones_mas_fuertes

cancer                            -0.272906
gastos_salud                      -0.262333
enfermedad_cardiaca               -0.242263
creatinina                        -0.087198
epoc                              -0.062929
hipertension                      -0.060034
diabetes                          -0.056181
leucocitos                        -0.052074
fumador                           -0.042654
glucosa                           -0.035429
colesterol                        -0.034901
edad                              -0.031451
actividad_fisica                   0.030640
obesidad                          -0.029257
ingresos_mensuales                 0.017886
ocupacion_Autónomo                 0.009621
trigliceridos                     -0.009379
tipo_empleo_Privado                0.009271
continente_nacimiento_Antártida   -0.007006
estado_civil_Viudo                -0.006747
deudas                            -0.006293
estado_civil_Casado                0.006252
tipo_empleo_Desempleado         

In [18]:
# quiero saber el numero de filas y columnas de df
print(f"El DataFrame tiene {df.shape[0]} filas y {df.shape[1]} columnas.")

El DataFrame tiene 50000 filas y 48 columnas.


In [19]:
y = df['vive']

X = df[['cancer', 'gastos_salud', 'enfermedad_cardiaca', 'creatinina', 'epoc', 'hipertension', 'diabetes', 
        'leucocitos', 'fumador', 'glucosa', 'colesterol', 'edad', 'actividad_fisica', 'obesidad', 'ingresos_mensuales', 
        'trigliceridos', 'tipo_empleo_Privado', 'estado_civil_Viudo', 
        'deudas', 'tipo_empleo_Desempleado', 'plaquetas', 'ocupacion_Jubilado', 'mut_BRAF', 'hemoglobina', 'mut_PIK3CA', 
        'mut_BRCA1', 'nivel_educativo', 'mut_ALK', 'tipo_empleo_Público', 'mut_TP53', 'mut_EGFR',
        'estado_civil_Soltero', 'continente_nacimiento_África']]

# División y SMOTE
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
smote = SMOTE(random_state=42)
X_train_smote, y_train_smote = smote.fit_resample(X_train, y_train)


model = tf.keras.Sequential([
    tf.keras.layers.Input(shape=(X_train.shape[1],)),
    tf.keras.layers.Dense(36, activation='relu',
        kernel_regularizer=regularizers.l2(0.001)),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(20, activation='relu',
        kernel_regularizer=regularizers.l2(0.001)),
    tf.keras.layers.Dropout(0.1),
    tf.keras.layers.Dense(10, activation='relu',
        kernel_regularizer=regularizers.l2(0.001)),
    tf.keras.layers.Dense(1, activation='sigmoid')
])



# Compilación
model.compile(optimizer="adam", loss='binary_crossentropy', metrics=['accuracy'])

# Callback de early stopping
early_stop = tf.keras.callbacks.EarlyStopping(
    monitor='val_loss',   # métrica que se observa
    patience=7,           # número de epochs sin mejora antes de parar
    restore_best_weights=True  # recupera los pesos del mejor epoch
)
#
# Entrenamiento con early stopping
history = model.fit(
    X_train_smote, y_train_smote,
    epochs=50,  # ahora puedes poner un número alto, el early stop lo detendrá antes
    batch_size=64,
    validation_split=0.2,
    callbacks=[early_stop],
    verbose=1
)

# Evaluación
test_loss, test_acc = model.evaluate(X_test, y_test)

Epoch 1/50
873/873 [==============================] - 2s 2ms/step - loss: 0.6252 - accuracy: 0.7078 - val_loss: 0.8136 - val_accuracy: 0.6665
Epoch 2/50
873/873 [==============================] - 2s 2ms/step - loss: 0.5660 - accuracy: 0.7442 - val_loss: 0.7801 - val_accuracy: 0.6658
Epoch 3/50
873/873 [==============================] - 2s 2ms/step - loss: 0.5499 - accuracy: 0.7480 - val_loss: 0.7517 - val_accuracy: 0.6801
Epoch 4/50
873/873 [==============================] - 2s 2ms/step - loss: 0.5407 - accuracy: 0.7512 - val_loss: 0.8046 - val_accuracy: 0.6730
Epoch 5/50
873/873 [==============================] - 2s 2ms/step - loss: 0.5345 - accuracy: 0.7532 - val_loss: 0.7417 - val_accuracy: 0.6765
Epoch 6/50
873/873 [==============================] - 2s 2ms/step - loss: 0.5298 - accuracy: 0.7561 - val_loss: 0.7183 - val_accuracy: 0.6749
Epoch 7/50
873/873 [==============================] - 2s 2ms/step - loss: 0.5256 - accuracy: 0.7597 - val_loss: 0.7291 - val_accuracy: 0.6827
Epoch 